# Sorting and Searching — Student Version
*Intro-to-Python lesson: stable sorting, sorting dicts vs. classes, and a careful build-up to binary search.*

## Part 1 — Warm-Up
Given the list `nums = [9, 1, 3, 7, 2, 8]`: ...

In [ ]:
# Part 1 — Warm-Up Code Cell
nums = [9, 1, 3, 7, 2, 8]
ascending = sorted(nums)
descending = sorted(nums, reverse=True)
print('ascending:', ascending)
print('descending:', descending)

# TODO: Insert 5 at the correct position in ascending
value = 5
idx = None  # TODO: determine position and insert

print('after inserting 5:', ascending)

## Part 2A — Using `sorted(key=...)`
We can sort using a key function. The key function receives each element and returns a sortable value.

In [ ]:
# Example: sort by length, then alphabetically
def word_length(word):
    return len(word)

words = ['at', 'to', 'pie', 'no', 'go', 'bee']
result = sorted(words, key = word_length)
print(result)

## Part 2B — Sorting Dictionaries & Custom Class with `__lt__`
Sorting a list of dictionaries requires specifying a key. A custom class can define `__lt__` to sort itself.

In [ ]:
# Sorting dictionaries with sorted(key=...)
students = [
    {'name': 'Chloe', 'gpa': 3.5, 'credits': 90},
    {'name': 'Ben', 'gpa': 3.7, 'credits': 60},
    {'name': 'Ava', 'gpa': 3.5, 'credits': 75},
]

def sort_gpa(dict):
    return dict['gpa']

def sort_gpa_name(dict):
    return dict['gpa'], dict['name']

sorted_by_gpa = sorted(students, key = sort_gpa)
sorted_by_two = sorted(students, key = sort_gpa_name)

print(sorted_by_gpa)
print(sorted_by_two)

What happens if we try to sort dictionaries with out a key?

In [ ]:
sorted(students)

### Custom Class Extending `dict`
Below: a class that behaves like a dictionary but defines its own sorting order using `__lt__`.

In [ ]:
class Student(dict):
    def __init__(self, name, gpa, credits):
        super().__init__(name=name, gpa=gpa, credits=credits)

    # Sort by GPA descending, then name ascending
    def __lt__(self, other):
        return (self['gpa'], self['name']) < (other['gpa'], other['name'])

objs = [
    Student('Ava', 3.9, 90),
    Student('Ben', 3.2, 60),
    Student('Chloe', 3.5, 75),
]

print(objs[0]['name'])
print()

for s in sorted(objs):
    print(s)

# Binary Search: Full Student–ChatGPT Simulation (3 Iterations + Final Version)

(includes diagrams + analogy)

## ✅ STUDENT (message 1)

>Hi ChatGPT — I’m in an intro Python class and we just learned merge sort and bubble sort.   
>Now I need to write my own binary search function, but I’m not sure where to start.   

>Could you guide me?
>I want to try writing it myself first — please don’t give me the answer right away.

## ✅ CHATGPT (response 1)

Absolutely — great way to learn.

Here’s how we’ll do this:

1. **You try a version.**
2. **I’ll point out exactly what’s wrong.**
3. **You fix it.**
4. **We repeat that for 3 rounds.**
5. **Final round:** once you get it working, I’ll show a professional version.

Here are a few gentle hints:

✅ The list must be **sorted**  
✅ Compare the target to the **middle**  
✅ Search **left** or **right** based on that  
✅ Always **shrink** the search range  
✅ Add a **base case** for "not found"  

Okay—go ahead!  
Write your first attempt.

---

## ✅ STUDENT (attempt 1)

```python
def binary_search(lst, target):
    mid = len(lst) // 2
    if lst[mid] == target:
        return mid
    elif target < lst[mid]:
        return binary_search(lst[:mid], target)
    else:
        return binary_search(lst[mid:], target)
```

## ✅ CHATGPT (feedback 1 — very detailed)

Good start — you captured the structure of binary search.

But there are **three major problems**:

### ✅ Issue 1 — No base case → infinite recursion

If the target does **not** exist, recursion never stops.

```python
if len(lst) == 0:
    return -1
```

In the better indexed version, the base case is:

```python
if low > high:
    return -1
```

### ✅ Issue 2 — Slicing the list is dangerous

Slicing copies data (extra time/memory) **and resets indices**.
That returns the **wrong index** for the original list.

**Why slicing breaks binary search:**

```
Original list:       [10, 20, 30, 40, 50]
Indices:              0   1   2   3   4
```

Searching for **50**:

1st call → mid=2 (value=30) → go right

```python
binary_search(lst[mid:], target)
```

Now the sliced list is:

```
[30, 40, 50]
indices: 0 1 2
```

More slicing eventually gives:

```
[50]
index=0
```

Your function returns: **0** (index in the tiny list),
but the correct index in the original list was **4**.


### ✅ Issue 3 — Right-half slice includes mid again

You wrote `lst[mid:]`, which **keeps mid** in the right half.  
That risks non-shrinking ranges or infinite recursion.  
  
---  
## ✅ STUDENT (attempt 2)

```python
def binary_search(lst, target, low=0, high=None):
    if high is None:
        high = len(lst)-1

    mid = (low + high) // 2

    if lst[mid] == target:
        return mid
    elif target < lst[mid]:
        return binary_search(lst, target, low, mid)
    else:
        return binary_search(lst, target, mid, high)
```

## ✅ CHATGPT (feedback 2)

Much better — you’re using `low` and `high` and not slicing.

But there are **two bugs** to fix:

### ✅ Issue 1 — Missing base case

Place this **before** computing `mid`:

```python
if low > high:
    return -1
```

### ✅ Issue 2 — Wrong ranges (must exclude mid)

Your recursive calls keep `mid` in range. Instead:

```
Left half:  high = mid - 1
Right half: low  = mid + 1
```
  
  
  
---

## ✅ STUDENT (attempt 3)

```python
def binary_search(lst, target, low=0, high=None):
    if high is None:
        high = len(lst) - 1

    if low > high:
        return -1

    mid = (low + high) // 2

    if lst[mid] == target:
        return mid
    elif target < lst[mid]:
        return binary_search(lst, target, low, mid - 1)
    else:
        return binary_search(lst, target, mid + 1, high)
```

## ✅ CHATGPT (feedback 3 — success!)

YES! ✅✅✅

This version:

✔ Handles `low/high` correctly
✔ Avoids slicing
✔ Moves `mid` properly (excludes it on the next call)
✔ Includes the correct base case
✔ Returns the correct index in the **original** list

The only improvement left is style and clarity — an iterative version is often preferred in practice.

---

## ✅ FINAL, REFACTORED VERSION (clean + commented)

```python
def binary_search(lst, target):
    """
    Return the index of target in sorted lst, or -1 if not found.
    Uses iterative binary search (often preferred to recursion).
    """
    low, high = 0, len(lst) - 1

    while low <= high:
        # Middle index
        mid = (low + high) // 2

        # Found it
        if lst[mid] == target:
            return mid

        # Search left half
        if target < lst[mid]:
            high = mid - 1
        # Search right half
        else:
            low = mid + 1

    # Not found
    return -1
```



In [ ]:
def binary_search(lst, target):   
    low, high = 0, len(lst) - 1
    while low <= high:        
        mid = (low + high) // 2       
        if lst[mid] == target:
            return mid        
        if target < lst[mid]:
            high = mid - 1       
        else:
            low = mid + 1    
    return -1
# Quick tests (students can run and add more)
nums = [2, 4, 6, 8, 10, 12, 14]
print(binary_search(nums, 10))  # expect 4
print(binary_search(nums, 5))   # expect -1


### Visualization — how iterative binary search shrinks the window

Example: searching for 50 in `[10, 20, 30, 40, 50]`

```


┌────────────────────────┐
│ [10, 20, 30, 40, 50]   │  low=0, high=4
│          ^             │  mid=2
└────────────────────────┘

           ↓ shrink right half

┌──────────────────┐
│ [40, 50]         │  low=3, high=4
│   ^              │  mid=3
└──────────────────┘

           ↓ shrink right half

┌──────────┐
│ [50]     │  low=4, high=4
│  ^       │  mid=4 → FOUND
└──────────┘

```

### Trace Table Template (fill in while testing)

| step | low | high | mid | lst[mid] | decision   |
|-----:|----:|-----:|----:|:--------:|:-----------|
|  1   |  0  |  4   |  2  |   30     | go right   |
|  2   |  3  |  4   |  3  |   40     | go right   |
|  3   |  4  |  4   |  4  |   50     | **found**  |


## Why Binary Search is Powerful
log₂(n) grows extremely slowly. Here is a plot for different values of n.

In [ ]:
import math
import matplotlib.pyplot as plt

ns = [10, 100, 1000, 10_000, 100_000, 1_000_000]
log2_ns = [math.log2(n) for n in ns]

plt.figure()
plt.plot(ns, log2_ns, marker='o')
plt.title("log2(n) across scales")
plt.xlabel("n")
plt.ylabel("log2(n)")
plt.margins(x=0.1)
plt.ylim(0, 20)
plt.show()

## Exercises
Complete these in class or as homework.

### Exercise 1 — Guessing Game
Create a number guessing game for numbers between 1 and 100.
Give feedback: too high, too low, correct.

In [ ]:
import random
secret = random.randint(1, 100)
attempts = 0

while True:
    guess = input("Enter a number (or q to quit): ")
    # TODO: implement guessing logic
    pass

### Exercise 2 — Trace Binary Search
Fill in the table 

### Binary Search Trace Table
Complete the trace for `lst = [2, 5, 7, 9, 12, 18, 21]` and `target = 12`:

| Step | low | high | mid | lst[mid] | Decision |
|-----:|----:|-----:|----:|---------:|:---------|
| 1 | | | | | |
| 2 | | | | | |
| 3 | | | | | |
| 4 | | | | | |

### Exercise 3 — Implement Linear & Binary Search
Implement both searches and compare runtime.

In [ ]:
import time, random

def linear_search(lst, target):
    # TODO
    return -1

def binary_search(lst, target):
    # TODO
    return -1

N = 500_000
data = list(range(N))
target = random.choice(data)

def time_search(fn, lst, target):
    start = time.perf_counter()
    result = fn(lst, target)
    return result, time.perf_counter() - start

lin_result, lin_time = time_search(linear_search, data, target)
bin_result, bin_time = time_search(binary_search, data, target)

print('Linear:', lin_result, lin_time)
print('Binary:', bin_result, bin_time)

### Exercise 4 —  
1) Sort Dicts by name  
2) create a sorted list of names  
3) search for a name 


In [ ]:
students = [
    {'name':'Ava','gpa':3.9,'credits':90},
    {'name':'Ben','gpa':3.2,'credits':60},
    {'name':'Chloe','gpa':3.5,'credits':75},
    {'name':'Dylan','gpa':3.8,'credits':45},
    {'name':'Ella','gpa':3.1,'credits':30},
]

#TODO

#print('Index found:', idx)

### Exercise 5 — Concept Questions
1) What is log₂(1e12)?
2) Why must binary search use sorted data?
3) When is linear search better?
4) Why is slicing inefficient in recursive binary search?
5) Worst-case runtime of binary search?
6) Best-case runtime?
7) Why use `low <= high`?
8) What happens if `mid` is miscomputed?